# Simulación de campaña — TelecomAI Customer Intelligence

Este notebook ilustra cómo las predicciones del modelo se traducen en KPIs de campaña para recomendar el plan **Ultra**.
Los números son **ejemplos ilustrativos** sobre datos sintéticos (`users_behavior.csv`).


In [ ]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import yaml

from main import load_config, build_model, build_preprocessor


In [ ]:
# Cargar configuración y datos
cfg = load_config(Path('configs/config.yaml'))
df = pd.read_csv(cfg.paths['data_csv'])
df.head()


In [ ]:
# Cargar pipeline entrenado (asumir make train ejecutado previamente)
model_path = Path(cfg.paths['model_export_path']) if 'model_export_path' in cfg.paths else Path('models/model_v1.0.0.pkl')
pipeline = joblib.load(model_path)

X = df[cfg.features]
y_true = df[cfg.target].to_numpy()
proba = pipeline.predict_proba(X)[:, 1]

# Threshold de decisión (actualmente 0.5 por defecto)
threshold = 0.5
y_pred = (proba >= threshold).astype(int)

n = len(df)
n_ultra = int(df[cfg.target].sum())
n_smart = int(n - n_ultra)
n, n_ultra, n_smart


In [ ]:
# Métricas de campaña
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Total clientes: {n}')
print(f'Clientes elegibles (is_ultra=1): {n_ultra}')
print(f'Contactados (pred=1): {tp + fp}')
print(f'TP (elegibles correctamente identificados): {tp}')
print(f'FP (no elegibles contactados): {fp}')
print(f'Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}')


In [ ]:
# KPIs económicos (supuestos ilustrativos)
uplift_arpu = 5.0   # USD/mes de ARPU adicional por cliente correctamente migrado a Ultra
contact_cost = 0.5  # USD de coste por contacto (SMS/call center)

saved_revenue = tp * uplift_arpu
contact_cost_total = (tp + fp) * contact_cost
net_gain = saved_revenue - contact_cost_total

print(f'Ingresos brutos por migraciones correctas: ${saved_revenue:,.0f}/mes')
print(f'Coste de campaña: ${contact_cost_total:,.0f}/mes')
print(f'Ganancia neta aproximada: ${net_gain:,.0f}/mes')


## Cohorts simples

Podemos analizar KPIs por cohortes (por ejemplo, alto vs bajo consumo de datos).


In [ ]:
# Definir cohortes por uso de datos
df_cohorts = df.copy()
df_cohorts['cohort'] = pd.qcut(df_cohorts['mb_used'], q=3, labels=['low_data', 'mid_data', 'high_data'])
df_cohorts['y_true'] = y_true
df_cohorts['y_pred'] = y_pred

rows = []
for cohort, sub in df_cohorts.groupby('cohort'):
    if len(sub) == 0:
        continue
    tn_c, fp_c, fn_c, tp_c = confusion_matrix(sub['y_true'], sub['y_pred']).ravel()
    prec_c = precision_score(sub['y_true'], sub['y_pred'])
    rec_c = recall_score(sub['y_true'], sub['y_pred'])
    rows.append({
        'cohort': str(cohort),
        'n_customers': len(sub),
        'precision': prec_c,
        'recall': rec_c,
        'tp': tp_c,
        'fp': fp_c,
    })

pd.DataFrame(rows)


### Notas

- Este notebook no forma parte del pipeline de entrenamiento; es una herramienta de exploración de impacto.
- Los valores de ARPU y coste de contacto son supuestos y deben ajustarse a la realidad del negocio antes de inferir ROI real.
- El threshold puede variarse para estudiar diferentes escenarios de campaña (por ejemplo, threshold=0.6 para más precisión, menos recall).
